In [ ]:
import synapseclient
import synapseutils


syn = synapseclient.Synapse()
syn.login()
# files = synapseutils.syncFromSynapse(
#     syn, 'syn52524005 ',
#     path='metabolics',
#     downloadFile=True
# )
files = synapseutils.syncFromSynapse(
    syn, 'syn52524002 ',
    path='metabolics',
    downloadFile=True
)
''

In [ ]:
import matplotlib
# matplotlib.use('PDF')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from pybeataml.load_data import AMLData

In [ ]:
from pybeataml.load_data import cluster_colors

In [ ]:
from statsmodels.stats.multitest import fdrcorrection

In [ ]:
from scipy.stats import fisher_exact
import numpy as np

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

matplotlib.rcParams['font.family'] = ['arial']
matplotlib.rcParams['font.size'] = 12

sns.set_theme(
    context ='paper',
    palette="Paired",
    style='white',
    font='arial',
    font_scale=1.0
)

In [ ]:
data = AMLData()

In [ ]:
data.clinical.columns.values

In [ ]:
data.meta['k=4']

In [ ]:
rename = {
            'positive': True,
            'negative': False,
            'y': True,
            'n': False,
            #np.nan: False
        }
data.clinical.replace(rename, inplace=True)
data.clinical

In [ ]:
cols = data.clinical.columns

In [ ]:
rna_patients = data.rna['sample_id'].unique()
np.savetxt('has_rna.csv', rna_patients, fmt='%s')

In [ ]:
data.flat_data.source.unique()

In [ ]:
# pivot_data = data.subset(['phospho', 'proteomics', 'rna_seq', 'wes',])
# pivot_data.set_index("sample_id",inplace=True)

In [ ]:
# pivot_data.to_csv("all_data_pivoted.csv.gz", compression='gzip')

In [ ]:
# mol_data = data.subset(['proteomics',  'rna_seq'])
# mol_data.head()

In [ ]:
cluster_labels = pd.read_csv('Data/nmf_rna_global_phospho_k_cluster_assignments.txt', delimiter='\t')
cluster_labels.head()

In [ ]:
data.clinical.head()

In [ ]:
#cols = ['FLT3-ITDcalls', 'NPM1calls', 'cumulativeChemo', 'overallSurvival']

In [ ]:
subtype_merged = cluster_labels.join(data.clinical.set_index('sample_id')).dropna(subset=['Barcode.ID'])
subtype_merged

In [ ]:
subtype_merged = data.meta[['k=4']].join(data.clinical.set_index('sample_id')).dropna(subset=['k=4'])
subtype_merged

In [ ]:
mutation_table = pd.pivot_table(
    data.wes.loc[data.wes.source=='wes'],
    columns='label', 
    index='sample_id', 
    values='exp_value',
    fill_value=0
)

In [ ]:
mutation_table = mutation_table.astype(bool)

In [ ]:
mutation_table.head()

In [ ]:
mutation_table.sum(axis=0).sort_values().tail(25)

In [ ]:
mutation_table = mutation_table.loc[:, (mutation_table.sum() > 5)].copy()

In [ ]:
subtype_merged = data.meta[['k=3', 'k=4', 'k=5', 'k=6', 'k=7', 'k=8']].join(data.clinical.set_index('sample_id')).join(mutation_table).dropna(subset=['k=4'])
subtype_merged.head()

In [ ]:
subtype_merged['NPM1_mut']

In [ ]:
mutation_table.shape

In [ ]:
mutation_cols = set(mutation_table.columns.values)
# mutation_cols.remove('label')
# mutation_cols.remove('sample_id')
len(mutation_cols)

In [ ]:
# melted_subtypes = subtype_merged.melt(id_vars=['Barcode.ID']+list(cols))
# melted_subtypes['n_subtypes'] = melted_subtypes['variable']
# melted_subtypes['n'] = melted_subtypes['value']
# del melted_subtypes['value']
# del melted_subtypes['variable']
# melted_subtypes

In [ ]:
cl = cluster_labels.drop('Barcode.ID', axis=1).copy()

In [ ]:
sns.clustermap(
    cl.sort_values('k=8'), figsize=(12,12),
    col_cluster=False,
    row_cluster=True,
    method='ward',
    cmap='Set2',
#     metric='hamming',
    # sns.color_palette("tab10")
    cbar_kws={"ticks":range(1,9)},
    cbar_pos=(1., .2, .03, .4),
    yticklabels=False,
);
plt.savefig("subtype_reoccurs_beataml.png", dpi=300, bbox_inches='tight')
plt.close()

In [ ]:
sns.clustermap(
    cl[['k=4', 'k=8']].sort_values(['k=4', 'k=8']), figsize=(12, 12),
    col_cluster=False,
    row_cluster=False,
    method='ward',
    cmap='Set2',
#     metric='hamming',
    # sns.color_palette("tab10")
    cbar_kws={"ticks": range(1, 9)},
    cbar_pos=(1., .2, .03, .4),
    yticklabels=False,
);
plt.savefig("subtype_reoccurs_beataml_4_8.png", dpi=300, bbox_inches='tight')
plt.close()

In [ ]:

var_cols = [
    'ageAtDiagnosis',
    
    'gender', 'FLT3-ITDcalls', 'NPM1calls', 'priorMalignancyNonMyeloid',
    'fabBlastMorphology','karyotype',
    
    'priorMalignancyType', 'cumulativeChemo',
    'priorMalignancyRadiationTx',
    'priorMDS', 'priorMDSMoreThanTwoMths',
    'priorMDSMPN', 'priorMDSMPNMoreThanTwoMths',
    'priorMPN',  'priorMPNMoreThanTwoMths', 
    
    'performanceStatus',
    'dxAtInclusion',
    'specificDxAtInclusion',
    'riskGroup',
    'otherCytogenetics', 'dxAtSpecimenAcquisition', 'specificDxAtAcquisition', 
    'ageAtSpecimenAcquisition', 
    'timeOfSampleCollectionRelativeToInclusion', 'specimenGroups', 'specimenType', 'ic50', 
    
    'priorTreatmentTypeCount', 'priorTreatmentTypes', 
    'priorTreatmentRegimenCount', 'priorTreatmentRegimens', 
    'priorTreatmentStageCount',  'priorTreatmentStages', 
    
    'responseToInductionTx', 
    'typeInductionTx', 'responseDurationToInductionTx', 
    
    'currentTreatmentType', 'currentRegimen', 'currentStage', 'currentTreatmentDuration', 
    'vitalStatus', 'overallSurvival', 'causeOfDeath', 
    'percentBlastsBM', 'percentBlastsPB', 'percentAbnormalPlasmaBM', 'percentBandsPB', 
    'percentBasophilsPB', 'percentEosinophilsPB', 'percentImmatureGranulocytesPB', 'percentLymphocytesPB',
    'percentMetamyelocytesPB', 'percentMonocytesPB', 'percentMyelocytesPB', 'percentNeutrophilsPB', 
    'percentNucleatedRBCsPB', 'percentPromonocytes', 'percentPromyelocytes', 'percentPromyelocytesPB', 
    'percentReactiveLymphocytesPB', 'percentWBC', 
    'wbcCount', 'plateletCount', 
    'albumin', 'bCellGeneRearrangement', 'bilirubin', 'creatinine', 'hemoglobin',
    'surfaceAntigensImmunohistochemicalStains', 'tCellReceptorGene',

]

num_cols = [
    'responseDurationToInductionTx',
    'ageAtSpecimenAcquisition',
    'timeOfSampleCollectionRelativeToInclusion',
    'currentTreatmentDuration',
    'overallSurvival',
    'ageAtDiagnosis'
]

for i in num_cols:
    print(i)
    var_cols.remove(i)

In [ ]:
non_useful = [
    'ic50',
    'dxAtInclusion',
    
    'percentBlastsBM',
    'percentAbnormalPlasmaBM',
    'percentBandsPB',
    'percentBasophilsPB',
    'percentEosinophilsPB',
    'percentImmatureGranulocytesPB',
    'percentLymphocytesPB',
    'percentMetamyelocytesPB',
    'percentMonocytesPB',
    'percentMyelocytesPB',
    'percentNeutrophilsPB',
    'percentNucleatedRBCsPB',
    'percentPromonocytes',
    'percentPromyelocytes',
    'percentPromyelocytesPB',
    'percentReactiveLymphocytesPB',
    'percentWBC',
    'percentBlastsBM', 'percentBlastsPB', 
    'percentAbnormalPlasmaBM', 'percentBandsPB', 
    'percentBasophilsPB', 'percentEosinophilsPB', 'percentImmatureGranulocytesPB', 'percentLymphocytesPB',
    'percentMetamyelocytesPB', 'percentMonocytesPB', 'percentMyelocytesPB', 'percentNeutrophilsPB', 
    'percentNucleatedRBCsPB', 'percentPromonocytes', 'percentPromyelocytes', 'percentPromyelocytesPB', 
    'percentBlastsPB',
    
    'albumin',
    'bCellGeneRearrangement',
    'bilirubin',
    'surfaceAntigensImmunohistochemicalStains',
    
    
    'creatinine', 'hemoglobin',
    'tCellReceptorGene'
]
for i in set(non_useful):
    var_cols.remove(i)

In [ ]:
subtype_merged[['priorMalignancyType']]

In [ ]:
to_str = [
    'priorTreatmentTypeCount',
    'priorTreatmentRegimenCount',
    'priorTreatmentStageCount'
    
]
for i in to_str:
    subtype_merged[i] = subtype_merged[i].astype(str)

In [ ]:
boolean_cols = [
    'FLT3-ITDcalls', 'NPM1calls',
] + list(mutation_cols)
for i in boolean_cols:
    subtype_merged[i] = subtype_merged[i].astype('boolean')

In [ ]:
subtype_merged[boolean_cols]

In [ ]:
def cat_enrichment(cluster_num_index, subtype_number, dataset, subtype, non_subtype,  variable_col):
    variables_to_check = dataset[variable_col].unique()
    results = []
    for i in variables_to_check:
        pos_for_subtype = subtype.loc[subtype[variable_col] == i].shape[0]
        neg_for_subtype =  subtype.shape[0] - pos_for_subtype
        
        pos_for_non_subtype = non_subtype.loc[non_subtype[variable_col]==i].shape[0]
        neg_for_non_subtype = non_subtype.shape[0] - pos_for_non_subtype

        table = [[pos_for_subtype, pos_for_non_subtype],
                 [neg_for_subtype, neg_for_non_subtype]]
        table = np.array(table)
        
        stat, p_val = fisher_exact(table, alternative='greater')
        results.append([cluster_num_index, subtype_number, variable_col+'_'+str(i), stat, p_val])
        
    results = pd.DataFrame(
        results, 
        columns=['cluster_num_index', 'subtype_number','variable_name', 'enrichment', 'p_value']
    )
    return results


def boolean_enrichment(cluster_num_index, subtype_number, subtype, non_subtype, variable_name):
    
    pos_for_subtype = subtype.loc[subtype[variable_name]].shape[0]
    neg_for_subtype =  subtype.shape[0] - pos_for_subtype
    
    pos_for_non_subtype = non_subtype.loc[non_subtype[variable_name]].shape[0]
    neg_for_non_subtype = non_subtype.shape[0] - pos_for_non_subtype
    table = [[pos_for_subtype, pos_for_non_subtype],
             [neg_for_subtype, neg_for_non_subtype]]
    table = np.array(table)
    stat, pval = fisher_exact(table, alternative='greater')
    return pd.DataFrame([[cluster_num_index, subtype_number, variable_name, stat, pval,]],
                        columns=['cluster_num_index', 'subtype_number', 'variable_name', 'enrichment', 'p_value'])



def calculate_enrichment(cluster_num_index, subtype_number, variable_name):

    subset_to_check = subtype_merged[[cluster_num_index, variable_name]].dropna(subset=[variable_name]).copy()

    if subset_to_check.shape[0] < 50:
        return pd.DataFrame()
    
    subtype = subset_to_check.loc[subset_to_check[cluster_num_index] == subtype_number]
    non_subtype = subset_to_check.loc[subset_to_check[cluster_num_index] !=subtype_number]
    
    col_type = subtype[variable_name].dtype
    if pd.api.types.is_bool_dtype(col_type):
        return boolean_enrichment(cluster_num_index, subtype_number, subtype, non_subtype, variable_name)

    elif pd.api.types.is_string_dtype(col_type):
        return cat_enrichment(cluster_num_index, subtype_number, subset_to_check, subtype, non_subtype, variable_name)
    else:
        print(variable_name)
        return pd.DataFrame()

def calculate_for_cluster_size(cluster_size, variable_name):
    results = []
    for i in subtype_merged[cluster_size].unique():
        results.append(calculate_enrichment(cluster_size, i, variable_name))
    return pd.concat(results, ignore_index=True)

def gather_enrichment(variables):
    results = pd.DataFrame()
    # for i in range(2, 10):
    for i in range(4, 5):
        k = f'k={i}'
        results_storage = [results]
        for var in variables:
            r1 = calculate_for_cluster_size(k, var)
            results_storage.append(r1)

        results = pd.concat(results_storage, ignore_index=True)
    return results

In [ ]:
subtype_merged['KRAS/NRAS'] = ((subtype_merged['KRAS_mut']) | (subtype_merged['NRAS_mut'])).copy()
subtype_merged['IDH1/IDH2'] = ((subtype_merged['IDH1_mut']) | (subtype_merged['IDH2_mut'])).copy()

In [ ]:
subtype_merged['specimenGroups']

subtype_merged  = pd.concat([subtype_merged, 
                             subtype_merged['specimenGroups'].str.get_dummies(sep = "|")], axis =1)

In [ ]:
subtype_merged['']

# subtype_merged  = pd.concat([subtype_merged, 
#                              subtype_merged['specimenGroups'].str.get_dummies(sep = "|")], axis =1)

In [ ]:
subtype_merged

In [ ]:
new_cols = ['Initial Acute Leukemia Diagnosis', 'Post-Chemotherapy', 'Post-Transplant', 
            'Relapse', 'Residual Disease', 'Residual Relapse',]
for i in new_cols:
    subtype_merged[i] = subtype_merged[i].astype(bool)

In [ ]:
var_columns = [
#     'FLT3-ITDcalls', 'NPM1calls', 'NPM1_mut',
#     'fabBlastMorphology', 
    'priorMalignancyType',
    'priorMalignancyNonMyeloid',
    'gender', 'cumulativeChemo',
    'priorMDSMPN', 'priorMDS',
#     'specimenGroups', 
#     'specimenType'
    
    
    # 'karyotype', 'riskGroup', 'priorTreatmentTypes'
]
var_columns += [ 'FLT3-ITDcalls', 'KRAS/NRAS', 'IDH1/IDH2'] + list(mutation_cols) + new_cols
results = gather_enrichment(var_columns)

In [ ]:
results.sort_values('p_value')

In [ ]:
results.p_value.shape

In [ ]:
for i, df in results.sort_values('cluster_num_index').groupby('cluster_num_index'):
    df = df.loc[df.p_value<0.05]
    if df.shape[0]>0:
        print(df.sort_values('subtype_number'))

In [ ]:
results['fdr_bh_sig'], results['fdr_bh'] = fdrcorrection(results.p_value)

In [ ]:
results.sort_values('fdr_bh')

In [ ]:
results.loc[results['fdr_bh_sig']]

In [ ]:
counts = subtype_merged.groupby(['k=4'])[list(mutation_cols)].sum().reset_index()
counts

In [ ]:
order = list(reversed(list(counts.sum().sort_values().index)))

In [ ]:
order.remove('k=4')

In [ ]:
counts_1 = counts.melt(id_vars='k=4')

In [ ]:
subtype_merged

In [ ]:
total_counts = subtype_merged.groupby(['k=4'])[list(mutation_cols)].sum().sum()
tc = pd.DataFrame(total_counts, columns=['value'])
tc.reset_index(inplace=True)
tc['variable'] = tc['index']

In [ ]:
norm_factor = subtype_merged.groupby('k=4').count()['gender'].to_dict()
for i in counts_1.variable.unique():
    crit = counts_1.variable==i
    to_div = tc[tc.variable==i].value
    counts_1.loc[crit, 'fraction'] = counts_1[crit].value/float(to_div)
for j in norm_factor:
    crit = counts_1['k=4']==j
    to_div = norm_factor[j]
    counts_1.loc[crit, 'fraction'] = counts_1[crit].value/float(to_div)*100

In [ ]:

half = int(len(order)/2)
top = order[:half]
s1 = counts_1.loc[counts_1.variable.isin(top)]
g = sns.catplot(
    data=s1, kind="bar",
    x="fraction", y="variable", hue="k=4", orient='horizontal',
    errorbar="sd", palette=cluster_colors, alpha=.9,
    height=6, aspect=.5,
    order=top,
)
g.set(xlim=(0, 50))
plt.savefig('mutation_freq_top.pdf', bbox_inches='tight', dpi=300)


bot = order[half:]
s2 = counts_1.loc[counts_1.variable.isin(bot)]
g = sns.catplot(
    data=s2, kind="bar",
    x="fraction", y="variable", hue="k=4", orient='horizontal',
    errorbar="sd", palette=cluster_colors, alpha=.9, 
    height=6, aspect=.5,
    order=bot,
)
g.set(xlim=(0, 50))
plt.savefig('mutation_freq_bot.pdf', bbox_inches='tight', dpi=300)

In [ ]:
counts_pivoted = counts_1.pivot(index='variable', columns='k=4', values='fraction').astype(float)
counts_pivoted

In [ ]:
piv_results = results.pivot(index='variable_name', columns='subtype_number', values='p_value').astype(float)
annotations = piv_results.copy()
annot = piv_results<0.05
annotations[annot] = '+'
annotations[~annot] = ''

In [ ]:
sns.clustermap(
    -1*np.log10(piv_results),
    annot=annotations,
    fmt='s',
    col_cluster=False,
    yticklabels=True,
    figsize=(4, 12),
    metric='correlation',
    cmap='Greens',
    linecolor='black',
    linewidths=0.003,
    cbar_kws=dict( label='$-log_{10}$(p-value)', use_gridspec=False,)
)

In [ ]:
to_keep = piv_results[piv_results<0.05].count(axis=1)
to_keep = to_keep[to_keep>0]
sig = piv_results.loc[to_keep.index]

In [ ]:
# sig.sort_values(order_cols, inplace=True)
order_cols = [1.0, 2.0, 3.0, 4.0]
ordered_set = []
for i in order_cols:
    tmp = sig[i].sort_values()
    for j in tmp[tmp<0.05].index:
        if j not in ordered_set:
            ordered_set.append(j)
print(ordered_set)
sig = sig.loc[ordered_set]

In [ ]:
sig.columns = sig.columns.astype(int)
sig.columns

In [ ]:
sig

In [ ]:
annotations = sig.copy()
annot = sig<0.05
annotations[annot] = 'x'
annotations[~annot] = ''

# Define two rows for subplots
fig, (ax, cax) = plt.subplots(ncols=2, figsize=(2, 5), 
                              gridspec_kw={"width_ratios":[1.0, .1]})

# Draw heatmap
# sns.heatmap(df, ax=ax, cbar=False)

# colorbar

g = sns.heatmap(
    -1*np.log10(sig),
    annot=annotations,
    fmt='s',
    yticklabels=True,

    ax=ax,

    cmap='Greens',
    linecolor='black',
    linewidths=0.01,
    cbar_kws={"orientation": "vertical", "pad": 0.05, "shrink": .82,
              'label':'$-log_{10}$(p-value)', 'use_gridspec':False},
    cbar=False,

)


fig.colorbar(ax.get_children()[0], cax=cax, orientation="vertical",
             label='$-log_{10}$(p-value)', pad = .5)

plt.savefig('subtype_enrichment.pdf', bbox_inches='tight')
plt.savefig('subtype_enrichment.png', bbox_inches='tight', dpi=300)

In [ ]:
wes = data.wes

In [ ]:
wes.loc[wes.gene_symbol=='FLT3']

In [ ]:
flt3_mut = wes.loc[wes.gene_symbol=='FLT3']
flt3_mut = flt3_mut.loc[flt3_mut.exp_value.astype(bool)]
flt3_mut.pivot(index='sample_id', values=['exp_value'], columns='label')